#Configurando

In [ ]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 41 kB/s 
     |████████████████████████████████| 198 kB 39.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=8da086d79952690d0a9e802185cb2e1b600d86d1b4970f11586b8c20f74b7c80
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [ ]:
import pyspark
import shutil
from pyspark.sql import SparkSession
from pyspark import SparkFiles #permite baixar arquivos

# criar uma sessão
spark = SparkSession.builder.appName('TDE2BD').getOrCreate()

# criar um contexto
sc = spark.sparkContext

# obtencao dos dados
base_url = 'http://www.ppgia.pucpr.br/~jean.barddal/bigdata/'
arquivos = ['transactions_sample.csv']

urls = [base_url + x for x in arquivos]
for url in urls:
  print(url)
  sc.addFile(url)

# Carregar os dados em um RDD
rdd = sc.textFile('file://' + SparkFiles.get('transactions_sample.csv'))

http://www.ppgia.pucpr.br/~jean.barddal/bigdata/transactions_sample.csv


#1. The number of transactions involving Brazil;

In [ ]:
# Ignorando o cabeçalho
rdd_sem_cabecalho = rdd.filter(lambda x: not x.startswith('country_or_area'))
rdd_sem_cabecalho.take(5)

['Afghanistan;2016;010410;Sheep, live;Export;6088;2339;Number of items;51;01_live_animals',
 'Afghanistan;2016;010420;Goats, live;Export;3958;984;Number of items;53;01_live_animals',
 'Afghanistan;2008;010210;Bovine animals, live pure-bred breeding;Import;1026804;272;Number of items;3769;01_live_animals',
 'Albania;2016;010290;Bovine animals, live, except pure-bred breeding;Import;2414533;1114023;Number of items;6853;01_live_animals',
 'Albania;2016;010392;Swine, live except pure-bred breeding > 50 kg;Import;14265937;9484953;Number of items;96040;01_live_animals']

In [ ]:
# para cada linha, verificar se a posição 0 é igual a Brasil
#se for, mantemos a linha, senão, ignoramos a linha

rdd_Brasil = rdd.filter(lambda l: l.split(";")[0] == 'Brazil')

In [ ]:
rdd_Brasil.take(5)
rdd_Brasil.count()

550

#2. The number of transactions per year;

In [ ]:
rdd_Anos = rdd_sem_cabecalho.map(lambda x: int(x.split(';')[1]))
rdd_Anos.take(5)
# criar um pair rdd nesse formato (ano, (numero de transacoes))

pairdd = rdd_Anos.map(lambda x: (x, 1))
pairdd.take(5)

[(2016, 1), (2016, 1), (2008, 1), (2016, 1), (2016, 1)]

In [ ]:
contagem = pairdd.reduceByKey(lambda x, y: x+y)
contagem.take(5)

[(2016, 354), (2008, 433), (2014, 499), (2012, 509), (2010, 474)]

#3. The most commercialized commodity (summing the quantities) in 2016, per flow type.

In [ ]:
# Ignorando o cabeçalho
rdd_sem_cabecalho = rdd.filter(lambda x: not x.startswith('country_or_area'))
rdd_sem_cabecalho.take(5)

['Afghanistan;2016;010410;Sheep, live;Export;6088;2339;Number of items;51;01_live_animals',
 'Afghanistan;2016;010420;Goats, live;Export;3958;984;Number of items;53;01_live_animals',
 'Afghanistan;2008;010210;Bovine animals, live pure-bred breeding;Import;1026804;272;Number of items;3769;01_live_animals',
 'Albania;2016;010290;Bovine animals, live, except pure-bred breeding;Import;2414533;1114023;Number of items;6853;01_live_animals',
 'Albania;2016;010392;Swine, live except pure-bred breeding > 50 kg;Import;14265937;9484953;Number of items;96040;01_live_animals']

In [ ]:
# filtra pelo ano de 2016
rdd_2016 = rdd.filter(lambda l: l.split(";")[1] == '2016')


In [ ]:
# para cada linha, verificar se a posição 1 é igual a 2016
#se for, mantemos a linha, senão, ignoramos a linha

#prddtupla = rdd_2016.map(montaTupla)
rdd_2016 = rdd_2016.filter(lambda x: x.split(";")[8] != "")
pairdd = rdd_2016.map(lambda x: ((x.split(";")[3], x.split(";")[4]), int(x.split(";")[8])))
somas = pairdd.reduceByKey(lambda x, y: x + y)

pairdd2 = somas.map(lambda x: (x[0][1], (x[0][0], x[1])))
maior = pairdd2.reduceByKey(lambda x, y: x if x[1] > y[1] else y)

maior.take(5)

[('Import', ('Animals, live, except farm animals', 15521160322)),
 ('Export', ('Fowls, live domestic < 185 grams', 148808219)),
 ('Re-Export', ('Sheep, live', 4900)),
 ('Re-Import', ('Horses, live except pure-bred breeding', 150))]

#4. The average of commodity values per year

In [ ]:
IX_YEAR = 1
IX_COMMODITY = 3
IX_PRICE = 5

def montaTuplaAvgComm(x):
  
  campos = x.split(";")

  year = campos[IX_YEAR]
  comm = campos[IX_COMMODITY]
  price = float(campos[IX_PRICE])

  yearComm = year + " " + comm

  return (yearComm, (price, 1))

prdd = rdd_sem_cabecalho.map(montaTuplaAvgComm)

In [ ]:
sum = prdd.reduceByKey(lambda x,y: (x[0] + y[0], x[1] + y[1]))

avg = sum.mapValues(lambda x: x[0]/x[1])

avg.take(5)

[('2008 Bovine animals, live pure-bred breeding', 11892512.942857143),
 ('2016 Swine, live except pure-bred breeding > 50 kg', 52862984.75),
 ('2016 Fowls, live domestic < 185 grams', 8461560.114285715),
 ('2016 Fowls, live domestic > 185 grams', 15575716.277777778),
 ('2016 Poultry, live except domestic fowls, > 185 grams', 1151334.2)]

#5. The average price of commodities per unit type, year, and category in the export flow in Brazil;

In [ ]:
prdd_filtro = rdd_sem_cabecalho.filter(lambda x: x.split(";")[0] == 'Brazil' and x.split(";")[4] == 'Export')

prdd_filtro.take(5)

prdd = prdd_filtro.map(lambda x: (x.split(";")[7] + ', ' + x.split(";")[1] + ', ' + x.split(";")[9], (float(x.split(";")[5]), 1)))

prdd.take(5)

soma = prdd.reduceByKey(lambda x,y: (x[0] + y[0], x[1] + y[1]))
soma.take(5)

avg = soma.mapValues(lambda x: x[0]/x[1])

avg.take(5)

[('Number of items, 2016, 01_live_animals', 28375283.4),
 ('Number of items, 2014, 01_live_animals', 92774914.625),
 ('Number of items, 2013, 01_live_animals', 86955326.77777778),
 ('Number of items, 2012, 01_live_animals', 58415213.72727273),
 ('Number of items, 2010, 01_live_animals', 77475318.1111111)]

#6. The commodity with the highest price per unit type and year.

In [ ]:
IX_YEAR = 1;
IX_PRICE = 5;
IX_UNIT_TYPE = 7;

# criar pair rdd (ano, (preço, unit_type))
def montaTuplaHighest(x):

  campos = x.split(";")

  year = campos[IX_YEAR]
  price = float(campos[IX_PRICE])
  unitType = campos[IX_UNIT_TYPE]

  yearUnitType = year + " " + unitType

  return (yearUnitType, price)

prdd = rdd_sem_cabecalho.map(montaTuplaHighest)

In [ ]:
highest = prdd.reduceByKey(max)
highest.take(5)

[('2014 Number of items', 1798687283.0),
 ('2013 Number of items', 1299072023.0),
 ('2012 No Quantity', 154905473.0),
 ('2011 Number of items', 873639517.0),
 ('2011 No Quantity', 353547579.0)]

#7. The number of transactions per flow type and year.


In [ ]:
prdd = rdd_sem_cabecalho.map(lambda x: ((x.split(";")[4], (x.split(";")[1])),1))

prdd.take(5)

somas = prdd.reduceByKey(lambda x,y: x + y)

somas.take(5)


[(('Export', '2016'), 140),
 (('Import', '2016'), 191),
 (('Import', '2015'), 298),
 (('Export', '2015'), 181),
 (('Import', '2014'), 296)]